In [ ]:
# !pip install numpy --upgrade
# !pip install mahotas

import os
import cv2
import glob
import random
import mahotas
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

from skimage.exposure import is_low_contrast
from skimage.filters import gabor_kernel
from scipy.stats import kurtosis, skew
from matplotlib import pyplot as plt
from skimage import exposure, feature, color
from skimage.feature import hog
from scipy import ndimage as ndi
from pywt import dwt2
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier


from crossdata_metrics import metrics, data_balanced, train_crossdataset



# Features extract

In [ ]:
kernels = []
for theta in range(2):
    theta = theta / 4. * np.pi
    for sigma in (2,3):
        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta,
                                              sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)

def gabor_feat(image):

    feats = np.zeros((len(kernels), 3), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats[k, 0] = filtered.mean() # Média
        feats[k, 1] = skew(filtered.flatten()) # Assimetria
        feats[k, 2] = kurtosis(filtered.flatten()) # Curtose
        _, (cH, cV, cD) = dwt2(filtered.T, 'db1')
        r_feat = np.hstack([feats[0],feats[1],feats[2],feats[3]])

    return r_feat 



# Momentos Zernike 
def ZernikeMoments(image):
    gray = image
    rows, cols = gray.shape
    radius = cols//2 if rows > cols else rows//2
    zernike = mahotas.features.zernike_moments(gray,radius)
    return zernike    


# Histogram of Oriented Gradients (HOG)
def fd_hog_gradients(image):

    fd, hog_image = feature.hog(image, orientations=8, pixels_per_cell=(128, 128),cells_per_block=(2, 2),
                                transform_sqrt=True , block_norm="L1",visualize=True)
    return fd


# LBP - Local Binary Pattern 
radius = 8
numPoints = 64
def describeLBP(image, eps=1e-7):
    lbp = feature.local_binary_pattern(image,numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, numPoints + 3),range=(0, numPoints + 2))
    return hist


# Load dataset


In [ ]:
def load_path(path):
    labels_ = os.listdir(path)
    features = []
    labels   = []
    for i, label in enumerate(labels_):
        cur_path = path + "/" + label 
        for image_path in glob.glob(cur_path + "/*"):
            img = cv2.imread(image_path)
            image = cv2.resize(img, (256,256))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            image2 = image/255
            lbp = describeLBP(image2) # LBP
            hog = fd_hog_gradients(image2) # HOG 
            zernike = ZernikeMoments(image2) # Momentos de Zernike
            gabor= gabor_feat(image2) # Filtros de Gabor 

            global_feature = np.hstack([hog,lbp, zernike, gabor])
            features.append(global_feature)
            labels.append(label)            
        print ("[INFO] completed label - " + label)
    return features, labels
### ACRIMA
ACRIMA_data, ACRIMA_labels = load_path("D:/Glaucoma/Banco_de_dados/ACRIMA/Images")

## REFUGE
REFUGE_data_train, REFUGE_labels_train = load_path("D:/Glaucoma/Banco_de_dados/REFUGE/cortes2")
REFUGE_data_test, REFUGE_labels_test = load_path("D:/Glaucoma/Banco_de_dados/REFUGE/REFUGE-Test400/Cortes_test")

#### RIM-ONE 
RO_data_train, RO_labels_train = load_path("D:/Glaucoma/Banco_de_dados/RIM-ONE_DL_images/partitioned_randomly/training_set")
RO_data_test, RO_labels_test = load_path("D:/Glaucoma/Banco_de_dados/RIM-ONE_DL_images/partitioned_randomly/test_set")


In [ ]:
%%time
# Refuge + Acrima + Rim-One DL
data = np.concatenate([ACRIMA_data, REFUGE_data_train, REFUGE_data_test, RO_data_train,RO_data_test], axis = 0)
labels_ = np.concatenate([ACRIMA_labels, REFUGE_labels_train, REFUGE_labels_test, RO_labels_train, RO_labels_test], axis = 0)

# Refuge + Acrima
data_re_ac = np.concatenate([ACRIMA_data, REFUGE_data_train, REFUGE_data_test], axis = 0)
labels_re_ac_ = np.concatenate([ACRIMA_labels, REFUGE_labels_train, REFUGE_labels_test], axis = 0)

# Rim-One + Acrima
data_ro_ac = np.concatenate([ACRIMA_data, RO_data_train,RO_data_test], axis = 0)
labels_ro_ac_ = np.concatenate([ACRIMA_labels, RO_labels_train, RO_labels_test], axis = 0)

# Rim-One + Refuge
data_ro_re = np.concatenate([REFUGE_data_train, REFUGE_data_test, RO_data_train,RO_data_test], axis = 0)
labels_ro_re_ = np.concatenate([REFUGE_labels_train, REFUGE_labels_test, RO_labels_train, RO_labels_test], axis = 0)

# Refuge
data_re = np.concatenate([REFUGE_data_train, REFUGE_data_test], axis = 0)
labels_re_ = np.concatenate([REFUGE_labels_train, REFUGE_labels_test], axis = 0)

# Acrima
data_ac = np.concatenate([ACRIMA_data], axis = 0)
labels_ac_ = np.concatenate([ACRIMA_labels], axis = 0)

# Rim-One
data_ro = np.concatenate([RO_data_train,RO_data_test], axis = 0)
labels_ro_ = np.concatenate([RO_labels_train, RO_labels_test], axis = 0)


le = LabelEncoder()
labels = le.fit_transform(labels_)
labels_re_ac = le.fit_transform(labels_re_ac_)
labels_ro_ac = le.fit_transform(labels_ro_ac_)
labels_ro_re = le.fit_transform(labels_ro_re_)
labels_re = le.fit_transform(labels_re_)
labels_ac = le.fit_transform(labels_ac_)
labels_ro = le.fit_transform(labels_ro_)

print("Data: ", np.shape(data))
print("Normal: ", len(data[labels==0]))
print("Glaucoma: ", len(data[labels==1]))


# Classify

In [ ]:
svm = SVC(C=3, kernel='rbf', probability=True)
mlp = MLPClassifier(activation='relu',max_iter=500,alpha=0.001,hidden_layer_sizes=(40,),learning_rate='constant')
xgb = XGBClassifier(colsample_bytree= 1, max_depth=5, n_estimators=300, subsample= 0.6, seed=0)

models = []
# models.append(('SVM', SVC(C=3, kernel='rbf')))
# models.append(('XGB', XGBClassifier(colsample_bytree= 1, max_depth=5, n_estimators= 100, subsample= 0.6, seed=0))) 
# models.append(('MLP', MLPClassifier(activation='relu',max_iter=1000,alpha=0.01,hidden_layer_sizes=(40,40,40),learning_rate='constant')))
models.append(('VOT', VotingClassifier(estimators = [('svm', svm),('mlp', mlp),('xgb', xgb)], voting='hard')))

# Results

## Unbalanced

In [ ]:
def results_unbalanced():
    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'None', models, 'scaler')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'None', models, 'scaler')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'None', models, 'scaler')
    return 

results_unbalanced()

## Random Balanced

In [ ]:
# Balanceamento aleatório
def results_balanced_random():

    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'random', models, 'scaler')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'random', models, 'scaler')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'random', models, 'scaler')
    
    return 
    
results_balanced_random()

## Near Miss Balanced

In [ ]:
# Balanceamento Near Miss
def results_balanced_nm():
    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'nm', models, 'scaler')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'nm', models, 'scaler')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'nm', models, 'scaler')
    return 
    
results_balanced_nm()

## Cluster Centroid Balanced

In [ ]:
# Balanceamento cluster

def results_balanced_cluster():
    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'cluster', models, 'scaler')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'cluster', models, 'scaler')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'cluster', models, 'scaler')
    
    return 
    
results_balanced_cluster()